# `PINT` Activity for IPTA 2019 Student Week $-$ Pune, India (NCRA/TIFR)
---

## Introduction

`PINT` (**P**INT **I**s **N**ot **T**empo3) is a modern software package for all-purpose timing of pulsars of all kinds across the electromagnetic spectrum.  It is written in `python` to be modular, user-friendly, and readable.  It is also compatible and in agreement with `tempo` & `tempo2`, and it plays well with the pulsar timing array toolkit `enterprise`.

For more details, documentation, and code, see Luo, J. et al. (soon-to-be-2019), https://nanograv-pint.readthedocs.io/en/latest/, and https://github.com/nanograv/PINT.

The data for this activity have been taken and _altered*_ from NANOGrav's 11-year data release (Arzoumanian et al. 2018, _ApJS_, 235, 37), which are available at http://nanograv.org/data.

_Activity credits: T. T. Pennucci.  Code snippets (mass function section) from E. Fonseca and M. T. Lam, with additional input and suggestions via J. Luo and the `PINT` code repository._

(* For one, the ECORR parameters have been removed so that GLS fitting can still be performed, which incorporates the other white noise parameters, but doesn't take too long to execute.)

---

## Overview

This activity assumes you are comfortable following along with an `ipython` notebook and that you have basic Astronomy 101 knowledge; detailed knowledge of pulsar timing is not a prerequisite, but any prior experience will help.

### In the first part of this activity, you will use `PINT` to load pulse times-of-arrival (TOAs), read in a timing model, fit the model to the TOAs, and explore residuals.

The pulsar you will examine is known as **J1614$-$2230**.  The name of the pulsar refers to its location on the sky in equatorial coordinates, approximately at 16 hours and 14 minutes in right ascension and -22 degrees and 30 arcminutes in declination, where the 'J' refers to the equinox and epoch of these coordinates, called 'J2000'.  J1614$-$2230 is a binary pulsar, meaning it has a companion star with which it resides in a gravitationally bound system.

You will learn some other characteristics of this pulsar throughout the activity.

### In the second part of this activitiy, you will update the timing model and make an astrophysical inference.

Pulsar timing often involves the repeated refinement of a model that accounts for how the neutron star spins, where it is located, how it is moving, how we are moving, material between us and the pulsar, and relativistic effects having to do with Special and General Relativity.

In practice, this bootstrapping process usually begins with making new timing observations of a pulsar, determining the TOAs from these observations, and then re-measuring the parameters in your timing model.  Sometimes it is necessary to add new parameters to the timing model.

The additional observations in the second part already come in the form of TOAs, and your job will be to ensure that the new timing model is an acceptable description of the data.

### Throughout the activity, you will be asked some simple questions; please enter your answers in the 'Markdown' (non-code) cells where indicated.

This is just for your own benefit, but please consult your peers to corroborate answers, or to ask for help.  If you cannot answer a question, that's OK, just move on.

---

## Tips & Troubleshooting

### Here are some quick reminders about `ipython` notebooks, in case you haven't use them much:
- To enter a code cell, just click it.  You can start typing straight away.
- To enter a 'Markdown' (non-code) cell, double-click it.  You can then start entering text.
- To exit a cell without executing it, press `Esc`.
- To execute any single cell (code or Markdown), once you have entered it, press `Ctrl`+`Enter` or push the Run button at the top of the page.
- Alternatively, you can run multiple cells by navigating to the proper entry in the `Cell` drop-down menu at the top of the page.
- To stop some code from continuing to run, you can press `Ctrl+C`, the square stop button, or navigate to `Kernel > Interrupt` in the drop-down menu at the top of the page.  You can also restart the entire activity by navigating to `Kernel > Restart & Clear Output`.
- If you want to add an empty cell to examine or experiment with something, you can push the `+` button at the top of the page, or navigate to `Insert > Insert Cell Below` in the drop-down menu at the top of the page; both will add an empty code cell below the currently active cell.  There are already some 'Sandbox' cells added for this purpose, to help answer the questions.
- To get help with a particular `python` function or object, you can type its name followed by `?` or `??` and execute the line.  In the first case you will get a description, in the second case you will get a description plus the code.  e.g., if you want to know more about the `len` function, you would execute `len??`.
- If you want to know what you can do with a `python` object, a handy trick within `ipython` is to type the name of the object, a period `.`, and then press `Tab`.  This will bring up a list of the attributes and methods of an object.  e.g., `my_object.`+`Tab` would display a list of attributes and methods for `my_object`.
- Similarly, `ipython` supports tab-completion, so pressing `Tab` in the middle of typing something will bring up a list of possible items you might want.
- To comment-out a line of python code, start the line with `#`; alternatively, you can change a code cell to be Markdown by selecting Markdown from the drop-down menu at the top of the page.
---

## _Please walk through this activity by executing cell-by-cell, not by running all cells at once!_
---

## Part I: Introduction to `PINT` and PSR J1614$-$2230 

In this section, you will get acquainted with some basic `ipython` functionality, `PINT`, and pulsar timing concepts, as well as the pulsar J1614$-$2230.

##### First import everything you will need, including the TOA, timing model, residual, and fitter modules from PINT:

In [ ]:
# imports
import pint.toa as toa  # module for handling input TOA data
import pint.models as models  # the collection of models that can be used in building a timing model
import pint.residuals as residuals  # this simple module subtracts the model from the data
import pint.fitter as fitter  # the underlying method of fitting the model to the data is a choice (modularity)
import astropy.units as units  # PINT objects come with an explicit unit, so you always know what the numbers mean
import numpy as np  # for doing numerical calcuations
import matplotlib.pyplot as plt  # for making and displaying plots
%matplotlib inline

In [ ]:
# set some default parameters for your plots
plotting_params = {'figure.figsize':(16.0,5.0),  # make the default plot size span the notebook width
                   'lines.markersize':5.0}  # make the default marker size a bit larger
plt.rcParams.update(plotting_params)

### Times-of-Arrival (TOAs)

A Time-of-Arrival (TOA) is a measurement of when a pulse of radiation has arrived at a telescope.  These measurements have already been made and serve as the input data to this pulsar timing activity.

##### Load the TOAs:

In [ ]:
# the first argument in the function below is the name of the file containing TOAs, which conventionally ends with
# '.tim', and is often referred to as a 'timfile'

# the second argument specifies which solar system ephemeris you'd like to use in making conversions/corrections;
#   here you will use NASA Jet Propulsion Laboratory (JPL)'s 'Development Ephemeris' (DE) 436

# the third argument tells PINT to use a pickle file for faster loading of TOAs

initial_toas = toa.get_TOAs("J1614-2230.init.tim", ephem="DE436", usepickle=True)

##### Get the Modified Julian Dates (MJDs) of the TOAs and the uncertainties on these measurements:

In [ ]:
initial_toa_mjds = initial_toas.get_mjds(high_precision=False)  # high precision values are not needed here
initial_toa_errs = initial_toas.get_errors()  # will be in units of microseconds [us]

A note about the TOA MJDs: these **_are_** the times-of-arrival, the MJD is the timestamp of the pulse arrival, but here you grab a low-precision version, just for plotting purposes (i.e., these are not used in fitting the timing model).

---

##### Examine the TOA table and summary:

In [ ]:
initial_toas_table = initial_toas.table  # everything about the TOAs is stored in a large table
initial_toas_table.show_in_notebook()  # though not too practical, this is one way to look at all the TOA info

In [ ]:
initial_toas_table.colnames  # each row is a TOA; here is the list of what the table contains for each TOA

In [ ]:
initial_toas_table['freq'][14]  # e.g., if you want to know the frequency [MHz] of the 15th TOA...

In [ ]:
initial_toas.get_freqs()[14]  # ... and to verify its unit of measurement, explicity index the TOAs

A note about units: `PINT` makes **_heavy_** use of units via the astropy.units module for transparency.  This means you may run into problems or `Error`s with some of python's other libraries (numpy, matplotlib, etc.) when doing mathematical operations if you're not careful.  Worst comes to worse, you can always append `.value` at the end of a numerical object to get a unit-free object, e.g.,:

In [ ]:
initial_toas.get_freqs()[14].value  # the same quantity, free of units, similar to grabbing it from the table

In [ ]:
print(initial_toas.get_summary())  # a more concise summary of the TOAs

##### An example of python indexing, get the table entries of TOAs with MJDs between 55000 and 57000:

In [ ]:
some_toas_table = initial_toas_table[initial_toa_mjds.value < 57000]  # TOAs earlier than 57000...
some_toas_table = some_toas_table[some_toas_table['mjd_float'] > 55000]  # ...from those, TOAs later than 55000
print(some_toas_table['mjd_float'].min(), some_toas_table['mjd_float'].max())  # check...

##### Examples of concise numpy calculations:

In [ ]:
print(np.sum(initial_toa_mjds.value > 56000))  # how many TOAs there are with MJDs > 56000, no units
print(np.mean(initial_toa_errs**-2, axis=-1)**-0.5)  # one formulation of the mean TOA uncertainty, with units

##### Sandbox cell:

#### Questions about the TOAs$-$

1. How many TOAs are there? [Hint: try exploring the attributes of initial_toas using the `.`+`Tab` trick mentioned earlier to find a specific attribute to answer this question.]

2. Which observatory/observatories do they come from?

3. What is the minimum and maximum TOA frequency in the dataset?

4. There are column headers in the TOA table referring to the position and velocity of the observatory relative to the Solar System Barycenter; what are typical values for these?  [Hint: don't forget to vector-add the three components!]  Is the distance similar to 1 AU?

5. What other interesting things can you learn about the TOAs from the table? [Hint: try looking at the 'flags' for one or more of the TOAs.]

---

### Timing Model

In a general sense, a Timing Model of a pulsar is a function that maps some coordinate time to the rotational phase of the neutron star.  For instance, it can take a TOA measured at the observatory and assign it an integer that corresponds to how many times the neutron star has rotated since an arbitrary start epoch.  A timing model can be as simple as a few numbers that describe the spin of the pulsar, or it can contain dozens of parameters to account for many systematic and stochastic components.

##### Load the initial timing model for J1614$-$2230:

In [ ]:
# the 'classic' format of a timing model is in the text format referred to as a 'parfile', ending with '.par'.
initial_model = models.get_model("J1614-2230.init.par")

Note that you are **_not_** generating a timing model 'from scratch' or from initial observations of this pulsar; you have a valid timing model solution to start with.  This is very important!  Solving for an initial timing model can be very tricky, indeed, and can take more than a year's worth of observations to get a good fit.

In [ ]:
# here is what the 'classic' parfile format looks like:
print(initial_model.as_parfile())

Without _a priori_ knowledge of pulsar timing, you wouldn't know what most of these parameters mean.  Broadly speaking, each line represents a parameter with the first column being the parameter name, followed by its value, a 1/0 flag indicating if it should be fit, and, finally, the uncertainty on the value.  For example, `F0` is the spin frequency of the pulsar, which is a fitted parameter with a value and uncertainty of 317.3789419569110(4) Hz.  That's quite a precise measurement!

In `PINT`, the timing model is divided up into _components_.  For example, one component of the timing model has to do with the _spin parameters_ of the neutron star, another component pertains to the _binary orbit_ (if applicable), yet another component deals with the _interstellar medium (ISM)_, and yet another represents our understanding of the motions of _the Earth and Solar System bodies_.  There is even a component to account for unmodeled, stochastic sources of uncertainty, generally referred to as _noise_.

##### List the components of the timing model:

In [ ]:
initial_model.components.keys()  # 'keys' are the labels of entries in a python dictionary; 'vals' are their counterparts.

##### These components fall into three broad categories within the `PINT` framework $-$ phase, delay, and noise components:

In [ ]:
initial_model.component_types

##### See how the components are divided up into these categories:

In [ ]:
print("Phase Components: %s\n"%map(type, initial_model.PhaseComponent_list))  # 'map' and 'type' are useful functions!

print("Delay Components: %s\n"%map(type, initial_model.DelayComponent_list))

print("Noise Components: %s\n"%map(type, initial_model.NoiseComponent_list))


##### The Phase Component is the most simple, which contains the spin of the neutron star; have a closer look:

In [ ]:
spindown_component = initial_model.components['Spindown']  # get the spindown component of the timing model
spindown_component = initial_model.PhaseComponent_list[0]  # equivalently, it's the first item in the list above
spindown_component.params  # list which of the parameters listed in the parfile are part of this component

In the above, `F0` is the spin frequency of the neutron star at the time `PEPOCH`, where `F1` is the first derivative of the frequency (which is usually negative, hence the term 'spindown').  If there were no other components to the timing model it would be simple to predict the observed spin frequency at any time $t$:
\begin{equation}
\textrm{F}(t) = \textrm{F0} + \textrm{F1} \times (t - \textrm{PEPOCH})
\end{equation}
Indeed, taking the integral of this equation would provide the rotational phase, and you would have a simple timing model.  However, the frequency `F` is really the spin frequency of the neutron star only if $t$ represents time in the inertial frame of the neutron star (modulo a constant Doppler shift from radial motion).  The observatory, which receives the pulse TOAs, is in a different, non-inertial frame of reference.  Therefore, you must apply many transformations $-$ or _delays_ $-$ to the TOAs to make it as though they were received in an inertial frame of reference.  These transformations make up the Delay Components of a timing model in `PINT`.

In order to track and predict the phase of rotation of the neutron star, you have to model all of these delays.  Of course, there is some arbitrary reference for 'phase zero', which is related to the AbsPhase component of the Phase Components.  Lastly, the PhaseJump part of the Phase Components refers to the constant phase offset between the phases of pulses arriving at difference frequencies $-$ something that you won't get to explore further here.

##### Take a look at one of the most important parts to Delay Components, the binary model:

In [ ]:
binary_model = initial_model.components['BinaryELL1']  # 'ELL1' refers to the exact binary model in use
binary_model.params  # the list of _all_possible_ parameters_ in this (ELL1) model

Here you will notice familiar Keplerian orbital parameters: `PB` is the orbital period, `A1` is the projected semi-major axis, `ECC` is the eccentricity, `OM` is the argument of periastron, and `T0` is the epoch of periastron passage.  The terms with `DOT` refer to their secular time derivatives, and the `EPS` parameters replace `E` and `OM` in very low eccentricity systems (like J1614$-$2230), in which case the time of ascending node `TASC` replaces `T0` as the epoch parameter.  If you're lucky, you can measure the inclination of the orbit via `SINI`, as well as the mass of the companion star `M2`.

##### If you want to see if one of these parameters is in the timing model, you can get the quantity by executing something like this:

In [ ]:
binary_model.PB.quantity  # the orbital period of the binary with units

##### But notice, not all of these parameters are included in the model:

In [ ]:
type(binary_model.OMDOT.quantity)  # is None; i.e., type() returns NoneType if it's not in the timing model

##### If you're not sure what one of these parameters is, you can get its description:

In [ ]:
binary_model.FB0.description  # FB parameters are used to describe Fast Binaries; FB0 is the inverse of PB

##### If the parameter is included, it can be fit for if it's not fixed at a certain value, if it's not 'frozen':

In [ ]:
binary_model.TASC.frozen  # is the binary phase epoch parameter TASC fixed in the timing model?

##### Now take a look at one aspect a local Delay Component, the Solar System Shapiro Delay:

In [ ]:
ss_shapiro = initial_model.components['SolarSystemShapiro']  # get the model component
ss_shapiro_delays = ss_shapiro.solar_system_shapiro_delay(initial_toas)  # calculate the delays at the TOA times
ss_shapiro_delays  # check the units

In [ ]:
# plot the Solar System Shapiro Delay at the TOA times
plt.plot(initial_toa_mjds, ss_shapiro_delays.to(units.us), 'kx')  # converting to microseconds [us] using .to(units.us)
ax1 = plt.gca()
ax1.set_xlabel('MJD [day]')
ax1.set_ylabel(r'Solar System Shapiro Delay [$\mu$s]')
ax2 = plt.twiny()
ax2.set_xlim(units.Quantity(ax1.get_xlim(),units.day).to(units.year).value + 1858 + 1) # +1 b/c there's no year 0!
ax2.set_xlabel('Year [yr]')

But what is this delay?  The _Shapiro Delay_, in general, is the additional amount of time that a signal takes to traverse a gravitational potential relative to zero potential.  The first-order effect, in essence, is the integrated gravitational time dilation effect: clocks nearer to a heavy object tick more slowly.

In the above calculation, only the effect of the Sun's gravitational potential is considered, but it is possible to examine the planetary contributions, too.  The periodicity in the plot should have an obvious interpretation.  The sharpness of the function reflects the fact that the source, in projection, reaches a small angular separation from the Sun once a year.  

##### Another important Delay Component arises from the dispersive property of the homogeneous, ionized, interstellar medium (ISM):

In [ ]:
DM_delay = initial_model.components['DispersionDM']  # a simple Dispersion Measure (DM) model
DM_delay.params  # list the parameters

The `DM` is the _dispersion measure_, which is the integral of the free electron density along the line of sight, and so has units of column density:
\begin{equation}
\textrm{DM} = \int n_e ~ dl
\end{equation}

You can see that this simple DM model has a constant `DM` at epoch `DMEPOCH` and a first derivative `DM1`.  However, you would find that this function is not implemented in the present timing model.  Rather, the 'DMX' DM model is used in this description of J1614$-$2230; DMX is a piecewise-constant model of the time-variable DM, which is used to account for short-timescale changes to the DM as a function of time.  This model is particularly appropriate for J1614$-$2230, as you have seen that it makes a close appraoch to the Sun, whose solar wind contributes to the measured DM.  If you look back at the printed parfile statement, you'll see lots of lines with 'DMX' $-$ that's a lot of model parameters!

Most importantly, the net effect of any non-zero dispersion measure is to delay radio signals of frequency $\nu$ relative to an infinite frequency signal (which would have no delay), by an amount proportional to the inverse of the frequency squared, such that _lower_ frequencies arrive _later_:
\begin{equation}
\Delta{t} \propto \textrm{DM} \times \nu^{-2}
\end{equation}

Next you'll see what the typical _relative_ dispersive delay is for the pulses in the dataset.

##### First, see what the distribution of TOA frequencies in the data set looks like:

In [ ]:
initial_toa_freqs = initial_toas.get_freqs()  # get the TOA frequencies, which you already saw to be in MHz

# make a histogram
counts, bins, patches = plt.hist(initial_toa_freqs.value, bins=50)
plt.xlabel('Frequency [MHz]')
plt.ylabel('Counts')

It looks like the TOAs come from observing the pulsar in two disparate radio frequency bands (you can confirm this by looking at the TOA flag `-fe`, which stands for 'frontend', a synonym for 'receiver'.  You will approximate the typical dispersive delay by using the middles of these two bands.

##### Calculate the dispersive delay between receiver bands:

In [ ]:
DM = DM_delay.DM.quantity  # get the DM, with units of [cm**-3 pc]
freq1 = units.Quantity(1500.0, 'MHz')  # give MHz units to the middle value of the first, higher frequency band
freq2 = units.Quantity(820.0, 'MHz')  # give MHz units to the middle value of the second, lower frequency band
DM_delay.dispersion_time_delay(DM, freq2) - DM_delay.dispersion_time_delay(DM, freq1)  # the relative dispersive delay

##### Sandbox cell:

#### Questions about J1614$-$2230 and its initial timing model$-$

1. What is J1614$-$2230's spin _period_?  How does it compare to the mean TOA uncertainty?  Would you think that indicates it is a 'good timer'?

2. What is the DM of J1614$-$2230?  If this source is ~1.2 kpc away, what's the mean electron density in the ISM along this line-of-sight?

3. How does the dispersive delay between the two frequency bands compare to the spin period?  What does this mean?

4. How big is the orbit of J1614$-$2230 around its companion relative to the radius of the Sun?

5. How big is the Solar System Shapiro Delay relative to the mean TOA uncertainty?  At what time of the year does J1614$-$2230 make its closest approach to the Sun on the sky?

6. You did not yet look a the last major component group to the timing model, the Noise Component.  List the model parameters in this component.  Taking a hint from the name of this noise model component, what do you think these parameters do?

7. Take a minute to think or discuss with a peer about why you need to know the positions and velocities of the planets, as well as how the Earth spins, for pulsar timing.  [Hint: in what kind of reference frame would it be best to observe TOAs?]

---

### Timing Residuals

In any science that involves modeling or prediction, residuals play a central role.  _Timing residuals_ are just the name for the residuals that arise in pulsar timing.

The timing model accounts for observed TOAs and predicts the arrival times of new pulses.  The timing residuals are simply the difference between the timing model values for TOAs and the observed TOA times.  Timing residuals are where all of the unmodeled effects lie $-$ be they signals of planetary companions, or gravitatitonal waves $-$ and so are of utmost importance in pulsar timing analyses.

`PINT` has a separate module that calculates residuals given input TOAs and a timing model $-$ think of it as a fancy calculator that only subtracts.

##### Form the 'pre-fit' residuals:

In [ ]:
initial_prefit_residuals = residuals.resids(initial_toas, initial_model)  # calculate the resids object
initial_prefit_residuals_us = initial_prefit_residuals.calc_time_resids().to(units.us)  # time residuals in [us]

If you answered Question 6 in the previous section, you may have noticed that the Noise Component has parameters that scale the TOA uncertainties.  You will need these to plot the residuals and to calculate some typical figures-of-merit.  Without going into detail, fortunately, `PINT` has a convenience function to calculate these.

##### Get the scaled TOA uncertainties:

In [ ]:
initial_toa_errs_scaled = initial_model.scaled_sigma(initial_toas)  # apply EFACs and EQUADs to TOA uncertainties

##### Plot the pre-fit residuals:

In [ ]:
# the argument to the plotting function fmt standard for 'format'; 'k' is black, and '.' is the dot marker for a point
plt.errorbar(initial_toa_mjds.value, initial_prefit_residuals_us, initial_toa_errs_scaled, fmt='k.')
ax1 = plt.gca()
ax1.set_xlabel('MJD [day]')
ax1.set_ylabel(r'Pre-fit Timing Residual [$\mu$s]')
ax2 = plt.twiny()
ax2.set_xlim(units.Quantity(ax1.get_xlim(),units.day).to(units.year).value + 1858 + 1)
ax2.set_xlabel('Year [yr]')

Ideal residuals will be white ('flat') and Gaussian distributed.

##### A classic metric to assess the timing quality is the weighted root-mean-square (WRMS) of the residuals:

In [ ]:
# a single-line WRMS calculation
prefit_WRMS = np.sqrt(np.sum((initial_prefit_residuals_us / initial_toa_errs_scaled)**2) / \
               np.sum(initial_toa_errs_scaled**-2))
print(prefit_WRMS)

If this is comparable to the TOA uncertainties, that's a good sign.  Notice that you did this calculation keeping the astropy units intact, but now walk through it step-by-step, to be clear.

In [ ]:
# a more fleshed-out WRMS calculation
weights = initial_toa_errs_scaled**-2  # the 'W' part of WRMS
vals = initial_prefit_residuals_us**2  # the 'S' part of WRMS
weighted_sum_vals = np.sum(weights * vals)
mean_vals = weighted_sum_vals / np.sum(weights)  # the 'M' part of WRMS
prefit_WRMS = np.sqrt(mean_vals)  # the 'R' part of WRMS

print(prefit_WRMS)

##### Another classic figure-of-merit for assessing the quality of the model is the so-called 'goodness-of-fit' or reduced chi-squared ($\chi_{\textrm{red}}^2$) value:

In [ ]:
prefit_chi2 = np.sum((initial_prefit_residuals_us / initial_toa_errs_scaled)**2)
print(prefit_chi2)  # notice this quantity is unitless, and is the same as weighted_sum_vals above

In [ ]:
# without fitting, the degrees-of-freedom is simply the number of data points (TOAs)
prefit_dof = initial_toas.ntoas - 1  # the d.o.f. need to be reduced by one (you always 'fit' the mean value)

prefit_chi2_reduced = prefit_chi2 / prefit_dof
print(prefit_chi2_reduced)

The TOA errors ought to be Gaussian '1-$\sigma$' uncertainties on the measurements, resulting in the common interpretation of $\chi_\textrm{red}^2$ as the average number of standard deviations per degree of freedom.  Ideally, you want this number to be 'close' to 1.0.

##### Sandbox cell:

#### Questions about the pre-fit residuals$-$

1. What is meant exactly by _pre-fit_ residuals?  What do you expect to see in pre-fit residuals?

2. Based on your calculations for the WRMS and $\chi_\textrm{red}^2$, do you think this is a good timing model to start with?  Why or why not?  If not, what might be 'wrong'?

3. One perpetual concern is the presence of 'outliers', which are individual measurements that fall outside of the model or statistical decription of the data and often show up in plots of timing residuals.  Does it look like any of the TOA data points are outliers?

4. If you have a look at the attributes of `initial_prefit_residuals` using the `.`+`Tab` trick, you would find that there are methods and attributes pertaining to `chi2`, `dof`, and `chi2_reduced`.  These numbers don't seem to agree with what you calculated.  Why might that be?  [Hint: you can use the `??` trick to investigate the code of the `get` functions.]

---

### Fitting a Timing Model to TOAs

Fitting a model to data is not particular to pulsar timing at all.  For this reason, `PINT` made the wise decision of making the fitter a modular choice, so that you can tailor your method of optimization to the data at hand.

Here, you will use `PINT`'s stock Generalized Least-Squares (GLS) fitter to see if the initial timing model is already a converged solution to the initial TOAs.

##### Instantiate the fitter and do the fit:

In [ ]:
# instantiation of a Class object
initial_fitter = fitter.GLSFitter(initial_toas, initial_model)  # give the fitter data (TOAs) and the model to fit

# do the fit, which returns the chi2 value
postfit_chi2 = initial_fitter.fit_toas(full_cov=True)  # full_cov=True will use the NoiseComponent; may take a while

##### Form the 'post-fit' residuals:

In [ ]:
# these are the new differences after the parameter values have been updated by the fit
initial_postfit_residuals = initial_fitter.resids
initial_postfit_residuals_us = initial_postfit_residuals.calc_time_resids().to(units.us)  # time residuals in [us]

##### Plot the post-fit residuals:

In [ ]:
plt.errorbar(initial_toa_mjds.value, initial_postfit_residuals_us, initial_toa_errs_scaled, fmt='k.')
ax1 = plt.gca()
ax1.set_xlabel('MJD [day]')
ax1.set_ylabel(r'Post-fit Timing Residual [$\mu$s]')
ax2 = plt.twiny()
ax2.set_xlim(units.Quantity(ax1.get_xlim(),units.day).to(units.year).value + 1858 + 1)
ax2.set_xlabel('Year [yr]')

These residuals appear to be very similar, but you will now quantify them to check that they are.

##### Calculate the post-fit WRMS:

In [ ]:
postfit_WRMS = np.sqrt(np.sum((initial_postfit_residuals_us / initial_toa_errs_scaled)**2) / \
               np.sum(initial_toa_errs_scaled**-2))
print(postfit_WRMS)

##### Calculate the post-fit $\chi_\textrm{red}^2$:

In [ ]:
postfit_chi2 = np.sum((initial_postfit_residuals_us / initial_toa_errs_scaled)**2)
postfit_dof = initial_postfit_residuals.dof  # now you use PINT's value for d.o.f., accounting for the fitted parameters
postfit_chi2_reduced = postfit_chi2 / postfit_dof
print(postfit_chi2_reduced)

These numbers are very similar, so you will directly plot the residuals against each other to be sure.

In [ ]:
# plot the pre-fit vs. post-fit residuals, and their difference
fig = plt.figure(figsize=(10,5))
ax1 = fig.add_axes((0.0, 0.0, 0.5, 1.0))
ax1.errorbar(initial_prefit_residuals_us, initial_postfit_residuals_us, initial_toa_errs_scaled,
             initial_toa_errs_scaled, fmt='k.')
xlim = ax1.get_xlim()
ax1.plot([xlim[0],xlim[1]],[xlim[0],xlim[1]], 'k:')  # line of equality
ax1.set_xlim(xlim)
ax1.set_ylim(xlim)
ax1.set_xlabel(r'Pre-fit Timing Residual [$\mu$s]')
ax1.set_ylabel(r'Post-fit Timing Residual [$\mu$s]')
ax2 = fig.add_axes((0.6, 0.0, 1.0, 1.0))
ax2.errorbar(initial_toa_mjds.value, initial_prefit_residuals_us - initial_postfit_residuals_us,
             initial_toa_errs_scaled, fmt='k.')
xlim = ax2.get_xlim()
ax2.plot([xlim[0],xlim[1]], np.zeros(2), 'k:')  # zero line
ax2.set_xlim(xlim)
ax2.set_xlabel('MJD [day]')
ax2.set_ylabel(r'Difference in Pre- and Post-fit Timing Residuals [$\mu$s]')
ax3 = plt.twiny()
ax3.set_xlim(units.Quantity(ax2.get_xlim(),units.day).to(units.year).value + 1858 + 1)
ax3.set_xlabel('Year [yr]')

In [ ]:
(initial_prefit_residuals_us - initial_postfit_residuals_us).max()  # maximum difference

In [ ]:
(initial_prefit_residuals_us - initial_postfit_residuals_us).min()  # minimum difference

In [ ]:
postfit_model = initial_fitter.model  # the post-fit model with updated parameter values

In [ ]:
print(postfit_model.as_parfile())

##### Sandbox cell:

#### Questions about the fit, post-fit residuals, and fitted timing model$-$

1. Based on the post-fit residuals, do you think the timing model had already converged?  Why or why not?

2. Do you think any of the model parameters in the fitted model were updated or changed in a significant way?

3. Quantify your answer to Question 2 by comparing the difference in pre- and post-fit values of some model parameters to their uncertainties.  Make sure to practice using the `ipython` interface to query the models (i.e., don't just look at the printed parfiles!).

4. Take a look at the arguments to the fit_toas() function; there is an option for maxiter, the maximum number of iterations to allow.  What is the default?  Since you only ran one iteration and saw such a small difference, would you expect additional iterations to change things significantly?  Try additional iterations, if you have time.

5. What do you expect to see when you add new data which may not be described perfectly by the model?  How will the various quantities change (residuals, WRMS, $\chi_\textrm{red}^2$, parameter values, etc.)?

---

## Part II: Updating J1614$-$2230's Timing Model

In this section, you will quickly combine a new TOA dataset with the initial one.  You will then examine the new pre-fit residuals, fit the model to the combined TOA dataset, examine the post-fit residuals, and will update the timing model with new parameters.

There will be fewer step-by-step instructions, as much of what you need is in the first part of the activity.

### The new, combined TOA dataset

##### First, combine the TOAs:

In [ ]:
# the exclamation mark '!' tells ipython to send this command to the linux shell for execution
# 'cat' concatenates the two timfiles and '>' redirects the output to the new file
! cat J1614-2230.init.tim J1614-2230.new.tim > J1614-2230.all.tim  # concatenate the initial and new timfiles

Usually, if 'new' TOAs come from the same observatory and the same observational setup (like these new TOAs do), you'd expect them to extend the timing baseline chronologically.  However, here the new TOAs emulate the common situation in which you might have two disparate observatories observing the same source over a similar period of time and you wish to combine the TOAs to get a more sensitive dataset $-$ this is the perennial issue in the IPTA!

Combining TOA datasets is **_never_** this easy in real life*, but for the sake of the exercise, it will be here.  In fact, `PINT` should have a better way to read in more than one timfile, or to extend a TOA list...!

(* Case in point, see Verbiest, J. P. W. et al. (2016) and Perera, B. B. P. et al. (soon-to-be-2019).)

##### Load the new, combined TOA dataset:

In [ ]:
all_toas =  # complete this line!  make sure to use the new TOA file, J1614-2230.all.tim

##### Don't forget to apply the Noise Model to the TOA uncertainties:

In [ ]:
all_toa_errs_scaled = # complete this line!

##### Sandbox cell:

#### Questions about the new, combined TOA dataset$-$

1. Go back to the previous section where you first examined the initial TOA dataset, repeat some of the same commands on the combined TOA dataset, and answer the same questions.

2. How is the combined TOA dataset different from the initial one?  How is it the same?

3. How many more TOAs are there?  Give both absolute and fractional values.  Would you expect to have to update the timing model based on this number?

---

### Check the pre-fit residuals

##### Form the combined pre-fit residuals:

In [ ]:
all_prefit_residuals =  # complete this line!
all_prefit_residuals_us =  # complete this line!

##### Plot the combined pre-fit residuals:

##### Calculate the new WRMS from the combined pre-fit residuals:

##### Calculate the new $\chi_\textrm{red}^2$ from the combined pre-fit residuals:

##### Sandbox cell:

#### Questions about the pre-fit residuals from the combined dataset$-$

1. How do your numbers compare to the residuals from the initial TOA dataset?  Why do you think they changed?

2. What do you think will be necessary to do next?

---

### Check the post-fit residuals

##### Fit the model to the combined dataset using the GLS fitter:

In [ ]:
all_fitter =  # complete this line!
all_postfit_chi2 =  # compelte this line!

##### Form the combined post-fit residuals:

In [ ]:
all_postfit_residuals =  # complete this line!
all_postfit_residuals_us =  # complete this line!

##### Plot the combined post-fit residuals:

##### Calculate the new WRMS from the combined post-fit residuals:

##### Calculate the new $\chi_\textrm{red}^2$ from the combined post-fit residuals:

##### Sandbox cell:

#### Questions about the post-fit residuals from the combined dataset$-$

1. Quantitatively speaking, how did the residuals 'improve' after fitting the same model to the combined dataset?

2. Do you think these new WRMS and $\chi_\textrm{red}^2$ values are acceptable?  What should you do next?

---

### Check residuals as a function of orbital phase

A quick glance at the residuals might indicate that some of the TOAs are outliers, but if you could examine the data closely, you would find that there is nothing wrong with them.  The correct thing to do is to update the timing model with new parameters.  But which parameters?  Well, one big hint will come from examining the residuals not as a function of MJD but of _orbital phase_.

##### Get the orbital phases at each TOA:

In [ ]:
# This step is not quite obvious from the API, so it's included for you
binary_model = all_fitter.model.components['BinaryELL1']
all_ophases = binary_model.binary_instance.orbits()  # This returns the TOAs' orbital phases
print all_ophases

In [ ]:
# The orbital phases are in units of cycles (1 cycle = 1 full orbit), but you will want to plot the residuals 'folded',
#   modulo one orbit, so you use the modulo operator to get the fraction of an orbit
all_ophases %= 1  # this syntax is the same as all_ophases = all_ophases % 1

##### Plot the pre- and post-fit residuals as a function of orbital phase on the same axes:

In [ ]:
plt.errorbar(all_ophases, all_prefit_residuals_us, all_toa_errs_scaled, fmt='k.', ms=2, alpha=0.5, label='Pre-fit')
plt.errorbar(all_ophases, all_postfit_residuals_us, all_toa_errs_scaled, fmt='r.', ms=2, alpha=0.5, label='Post-fit')
plt.xlim(0.0,1.0)
plt.xlabel('Orbital Phase [cycle]')
plt.ylabel(r'Pre- & post-fit Timing Residual [$\mu$s]')
plt.legend(loc=2, frameon=False)

##### Sandbox cell:

#### Questions about the combined pre- and post-fit residuals as a function of orbital phase$-$

1. If you're unsure about it, convince yourself that you understand the modulo arithmetic (%) in getting the 'folded' orbital phases!  Try some simple calculations with decimal numbers.

2. Reconsider your answer to Question 1 from the previous section; how did the post-fit residuals 'improve' over the pre-fit residuals?  Does it look like there is something in the data that the model cannot 'hide'?

3. The peaked signal at orbital phase 0.25 should remind you of the Solar System Shapiro Delay you plotted earlier.  Taking that hint, what do you think you are looking at now?  What configuration of the binary orbit do you think the two stars are in at orbital phase 0.25?  What parameters might you need to add, to improve the timing model?

---

### Update the timing model

#### Shapiro Delay

Indeed, you will need to model the _binary's_ Shapiro Delay as part of the timing model.  If you look back at the `ELL1` binary model parameters, you will see the parameters `M2` (the companion star's mass $m_{\textrm c}$ in units of Solar masses) and `SINI` (the $\sin$ of the inclination $i$ of the orbit).  These two parameters parameterize the Shapiro Delay for _low-eccentricity orbits_ (like J1614$-$2230) in the following way:

\begin{equation}
r = m_{\textrm c} T_\odot, \\
s = \textrm{sin}~i, \\
\Delta_{\textrm{SD}} = -2~r~\textrm{ln}(1 - s~\textrm{sin}~\Phi), \\
\end{equation}

where $\Phi$ is the orbital phase, $T_\odot = G~{\textrm M}_\odot/c^3 = 4.925490947~\mu\textrm{s}$ is the 'mass of the Sun ${\textrm M}_\odot$ in units of time', and $G$ and $c$ are the familiar Newton's gravitational constant and the speed of light, respectively.  $r$ and $s$ are known as the 'range' and 'shape' parameters of the Shapiro Delay, with the clear intepretation that a larger mass linearly increases the delay and a higher inclination 'sharpens' the shape at superior conjunction.  However, this parameterization is not always good, and you are encouraged to read Freire & Wex (2010) for more details.

#### Mass function

In order to get a reasonable fit without exploring the entire space of the likelihood function, you will have to make a good guess for $m_{\textrm c}$.  If you're familiar with binary systems, you'll know that this is part of the _mass function ($f_m$)_:

\begin{align}
    f_m &= \frac{4\pi^2x^3}{T_\odot P_{\textrm b}^2} = \frac{(m_{\textrm c}\sin i)^3}{(m_{\textrm p} + m_{\textrm c})^2},
\end{align}

where $P_{\textrm{b}}$ is the orbital period (`PB`), $x$ is the projected semi-major axis (`A1`) of the pulsar's orbit, and $m_{\textrm p}$ is the pulsar's mass in Solar masses.  Notice that you can always measure the terms in the middle (and, therefore, $f_m$) with usual pulsar timing of binary systems.  With two additional assumptions, you can make an educated guess for $m_{\textrm c}$.  Theoretical considerations indicate that $m_{\textrm p} \approx 1.4\textrm{ M}_\odot$ for most neutron stars, and empirical distributions bear this out, with a second distribution with a slightly higher mean.  If you assume $\sin i = 1$ (i.e., that the orbit is edge-on), this will give us a _lower limit_ on $m_{\textrm c}$, call it the 'minimum companion mass', $m_{\textrm c, min}$.  Rearranging the above equation, 

\begin{equation}
    m_{\textrm c, min}^{3/2} - f_m^{1/2}(m_{\textrm p} + m_{\textrm c, min}) = 0.
\end{equation}

This equation cannot be solved analytically to obtain $m_{\textrm c, min}$, so you will use a function imported from the pulsar search software `PRESTO` that numerically finds an acceptable solution.  Notice that the answer will not depend sensitively on $m_{\textrm p}$ because of a shallow linear dependence.  Also note that in order to see 'obvious' evidence of Shapiro Delay in the residuals as you have, the inclination must already be high, so you can be reasonably sure that `M2` is close to $m_{\textrm c}$.

##### Get the minimum companion mass:

In [ ]:
import psr_utils as pu  # from PRESTO
M2_min = pu.companion_mass_limit(binary_model.PB.value, binary_model.A1.value, mpsr=1.4)  # will have units of M_sun
print(M2_min)

##### Since the model was already fit to the data, reset it before adding new parameters:

In [ ]:
all_fitter.reset_model()  # reset the model to have the initial parameters
binary_model = all_fitter.model.components['BinaryELL1'] # regrab the binary model

##### Add new Shapiro Delay parameters to the binary model:

In [ ]:
binary_model.M2.quantity = M2_min  # NB: this is one way in PINT to set a model parameter at a certain value!
binary_model.SINI.quantity = 0.9999  # This choice is unfortunately _very_ sensitive, which you won't explore here
binary_model.M2.frozen = binary_model.SINI.frozen = False  # allow the parameters to vary in the fit

A note on the choice for the initial `SINI` value: as mentioned earlier, you can expect the inclination to be quite high ($i > 80^{\circ}$, corresponding to `SINI` > 0.99) because you can see the Shapiro Delay in the residuals by eye.  Your estimate for `M2` came from assuming that `SINI` $\approx$ 1.0, so you can use a value nearly in agreement that isn't a boundary value.  This is a cheap way of getting around the issue of non-linearities in fitting Shapiro Delay!

##### Fit the extended timing model to the combined TOA dataset:

In [ ]:
all_fitter.fit_toas(full_cov=True)  # a final fit

##### Retrieve the final model and orbital phases:

In [ ]:
final_model = all_fitter.model  # get the new model
final_postfit_residuals = all_fitter.resids  # get the new post-fit residuals
final_postfit_residuals_us = final_postfit_residuals.calc_time_resids().to(units.us)  # time residuals in [us]

binary_model = final_model.components['BinaryELL1']  # the binary portion of the final model
final_ophases = binary_model.binary_instance.orbits() % 1  # get the orbital phases, modulo 1

##### Plot the final post-fit residuals as a function of orbit:

##### Plot the final post-fit residuals as a function of MJD:

##### Calculate the final WRMS value:

##### Calculate final $\chi_\textrm{red}^2$ value:

##### Write out your parfile for later

In [ ]:
with open('J1614-2230.final.par', 'w') as outfile:
    outfile.writelines(final_model.as_parfile())
outfile.close()  # Don't forget to check it later against Arzoumanian et al. (2018)!

##### Make a final plot to see the full glory of the Shapiro Delay detection:

In [ ]:
# define a convenience function to calculate the Shapiro Delay
def shapiro_delay(ophases, r, s):  # surely this is in PINT somewhere already!
    return -2 * r * np.log(1 - s*np.sin(ophases))

In [ ]:
# first get the Shapiro Delay parameters:
final_M2 = binary_model.M2.value
final_SINI = binary_model.SINI.value

# then set the Shapiro Delay parameters to zero (effectively remove it from the model):
binary_model.M2.quantity = 0.0
binary_model.SINI.quantity = 0.0

# calculate the residuals such that the residuals contain the full Shapiro Delay signal:
final_postfit_residuals_with_SD_us = residuals.resids(all_toas, all_fitter.model).calc_time_resids().to(units.us)
final_postfit_residuals_with_SD_us -= final_postfit_residuals_with_SD_us.mean()  # take out the constant offset residual

# now calculate the Shapiro Delay for the best-fit parameters:
ophases = np.linspace(0,2*np.pi,1000)  # evaluate it at 1000 orbital phases
r = 4.925*final_M2  # the 'range' parameter
s = final_SINI  # the `shape` parameter
SD = shapiro_delay(ophases, r, s)  # the Shapiro Delay
SD -= SD.mean()  # take out the constant offset delay

In [ ]:
# plot it!
plt.errorbar(final_ophases, final_postfit_residuals_with_SD_us, all_toa_errs_scaled, fmt='k.', ms=2, alpha=0.5,
             label='data')
plt.plot(ophases/(2*np.pi), SD, 'r--', lw=2, label='model')
plt.xlim(0.0,1.0)
plt.xlabel('Orbital Phase [cycle]')
plt.ylabel(r'Post-fit Timing Residual [$\mu$s]')
plt.legend(loc=2, frameon=False)

---
### Infer the pulsar mass

Looking back at the equation for the mass function, which you already know, once $m_{\textrm c}$ and $\sin i$ are measured, you can solve for the pulsar mass:

\begin{align}
    m_{\textrm{p}} = \frac{(m_{\textrm c}\sin i)^{3/2}}{f_m^{1/2}} - m_{\textrm c}
\end{align}

`PRESTO`'s psr_utils again will provide the convenience function for this calculation.

##### Estimate the pulsar mass:

In [ ]:
final_PB = binary_model.PB.value  # the final orbital period [d]
final_A1 = binary_model.A1.value  # the final projected semi-major axis [ls]
final_i = np.arcsin(final_SINI) * 180 / np.pi  # the final orbital inclination [deg]
pulsar_mass = pu.pulsar_mass(final_PB, final_A1, final_M2, final_i)  # another convenience function from PRESTO
print("J1614$-$2230 has a mass of %.2f Solar masses!"%pulsar_mass)

At this point, you would hope to publish.  However, you should first read Cromartie et al. (2019) :)

##### Sandbox cell:

#### Questions about updating the timing model and measuring Shapiro Delay$-$

1.  Are the final parameter values physical?  How do they compare with the _initial_ timing model parameters?  Which parameters changed significantly?

2.  See how your final parameters compare with those published in Arzoumanian et al. (2018).  In particular, what are the uncertainties?

3.  Do you think your final WRMS and $\chi_\textrm{red}^2$ values are acceptable?  What subtle aspect(s) of the timing model (or TOA dataset) could be improved?

4.  If you change the initial `M2` and `SINI` values in the final fit, what happens?  What might be the problem here?  [Hint: try plotting the Shapiro Delay for shallow inclinations and see if the shape reminds you of anything.]

---
# Good work! $-$ The end!
---

## ...here be dragons...

## Part III: Properly estimating the Shapiro Delay and its uncertainties via MCMC

As it turns out, the estimatation in Part II of the Shapiro Delay parameters was a bit finessed, and the uncertainties would have to be checked via Monte Carlo methods.  Fortunately, `PINT` can handle this, too!  Using an external sampler from `emcee`, we can get at the posterior distribution of the parameters and better estimate their uncertainties and covariances by looking at the chain of samples.

**However, this part of the activity has not been fully developed or tested, so explore it at your own risk/leisure.**

##### You will need the special `mcmc_fitter` module and the sampler from the separate software package `emcee`:

In [ ]:
# additional imports
import pint.mcmc_fitter as mcmc_fitter
from pint.sampler import EmceeSampler

##### Load the full TOA data set and the final model from Part II:

In [ ]:
all_toas = toa.get_TOAs("J1614-2230.all.tim", ephem='DE436', usepickle=True)
final_model = models.get_model("J1614-2230.final.par")

##### Fix all components, except the Shapiro Delay (and absolute phase):

In [ ]:
for component in final_model.components.keys():
    for param in final_model.components[component].params:
        exec('final_model.components[component].%s.frozen = True'%param)  # freeze everything
final_model.M2.frozen = final_model.SINI.frozen = False  # but allow the Shapiro Delay parameters to vary in the fit

##### Additionally, put a uniform prior on `SINI` that is zero outside of the range of the $\sin$ function [-1.0, 1.0]:

In [ ]:
final_model.SINI.prior = models.priors.Prior(models.priors.UniformBoundedRV(-1.0,1.0))

##### Define a log-likelihood function (see `PINT`'s examples it its `git` repository):

In [ ]:
def lnlikelihood_chi2(ftr, theta):  # ftr is the fitter, theta is the vector of parameters
    ftr.set_parameters(theta)
    #Uncomment to view progress
    #print('Count is: %d' % ftr.numcalls)
    return -residuals.resids(toas=ftr.toas, model=ftr.model).chi2.value  # proportional to the negative log-likeilhood

##### Initialize the emcee sampler:

In [ ]:
sampler = EmceeSampler(nwalkers=100)  # see emcee documentation to choose nwalkers intelligently

##### Initialize the fitter:

In [ ]:
fitter_mcmc = mcmc_fitter.MCMCFitter(all_toas, final_model, sampler, lnlike=lnlikelihood_chi2)

##### Sample and analyze at your own risk:

In [ ]:
#all_fitter_mcmc.fit_toas()  # fit the model (here just the Shapiro Delay parameters) to the TOAs

##### Sandbox cell:

#### Questions about MCMC in `PINT`$-$

TBD

---